In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# =========================================
# 📦 Import Libraries
# =========================================
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import psycopg2
from sqlalchemy import create_engine, text

In [ ]:
# ============================
# ⚙️ Set Database Connection
# ============================
engine = create_engine('postgresql+psycopg2://postgres:postgres@/mimic?host=/var/run/postgresql')

try:
    with engine.connect() as conn:
        # Get count
        result = conn.execute(text("SELECT COUNT(*) FROM mimiciii.patients"))
        count = result.scalar()
        print(f"✅ Connected! Patients table has {count} records.")
        # Load into pandas
        df_patients = pd.read_sql("SELECT * FROM mimiciii.patients LIMIT 5;", conn)
        display(df_patients.head(5))

except Exception as e:
    print("❌ Failed to connect or query:", e)


✅ Connected! Patients table has 46520 records.


,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,234,249,F,2075-03-13,NaT,NaT,None,0
1,235,250,F,2164-12-27,2188-11-22,2188-11-22,None,1
2,236,251,M,2090-03-15,NaT,NaT,None,0
3,237,252,M,2078-03-06,NaT,NaT,None,0
4,238,253,F,2089-11-26,NaT,NaT,None,0


In [ ]:
# ==========================
# 📥 Load Structured Data
# ==========================
query_structured = """
SELECT icustay_id,
       MAX(CASE WHEN itemid = 223900 THEN valuenum ELSE NULL END) AS pain_score,
       MAX(CASE WHEN itemid = 225309 THEN 1 ELSE 0 END) AS restraints_used,
       MAX(CASE WHEN itemid = 226733 THEN 1 ELSE 0 END) AS family_meeting
FROM mimiciii.chartevents
WHERE itemid IN (223900, 225309, 226733)
GROUP BY icustay_id;
"""

try:
    # Run the query and load into pandas DataFrame
    print(f"✅ connected...")
    structured = pd.read_sql(query_structured, engine)
    display(  display(structured.head(5)))

except Exception as e:
    print("❌ Failed to execute structured data query:", e)


✅ connected...


,icustay_id,pain_score,restraints_used,family_meeting
0,200001.0,5.0,0,0
1,200010.0,5.0,0,0
2,200011.0,5.0,0,0
3,200016.0,5.0,1,0
4,200021.0,5.0,0,0


None

In [ ]:
# ========================
# 📥 Load Clinical Notes
# ========================
query_notes = """
SELECT i.icustay_id, n.notes
FROM mimiciii.icustays i
JOIN (
    SELECT hadm_id, STRING_AGG(text, ' ') AS notes
    FROM mimiciii.noteevents
    WHERE category IN ('Nursing', 'Physician')
    AND iserror IS NULL
    GROUP BY hadm_id
) n ON i.hadm_id = n.hadm_id;

"""

try:
    # Run the query and load into pandas DataFrame
    notes = pd.read_sql(query_notes, engine)
    # Display only top 5 rows
    display(notes.head(5))

except Exception as e:
    print("❌ Failed to execute clinical notes query:", e)


,icustay_id,notes
0,209281,"59 year old male with HCV-related cirrhosis, g..."
1,269533,Acute Pain\n Assessment:\n pt c/o back and...
2,282580,58yom with terminal progressive MS since [**21...
3,270105,"Chief Complaint: Pancytopenia, ICH\n HPI:\n..."
4,221136,"Chief Complaint: Pancytopenia, ICH\n HPI:\n..."


In [ ]:
# =========================================
# 🛠️ Merge and Preprocess Data
# =========================================
df = structured.merge(notes, on='icustay_id')
scaler = StandardScaler()
df[['pain_score', 'restraints_used', 'family_meeting']] = scaler.fit_transform(
    df[['pain_score', 'restraints_used', 'family_meeting']]
)

In [ ]:
# ================================
# Extract ClinicalBERT Embeddings
# ================================
print("Loading ClinicalBERT...")
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

def get_note_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

print("Generating embeddings...")
df['bert_embedding'] = df['notes'].apply(lambda x: get_note_embedding(str(x)))

# Split embedding into separate columns
bert_df = pd.DataFrame(df['bert_embedding'].tolist(), index=df.index)
bert_df.columns = [f'bert_{i}' for i in range(bert_df.shape[1])]
df = pd.concat([df, bert_df], axis=1)


Loading ClinicalBERT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating embeddings...


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
# =======================================
# Define Mistrust Labels (for training)
# =======================================
df['noncompliant_label'] = df['notes'].str.contains('refused|noncompliant|left AMA', case=False, na=False).astype(int)


In [ ]:
# =======================
# Train Ensemble Models
# =======================

# Define features
features = ['pain_score', 'restraints_used', 'family_meeting'] + list(bert_df.columns)
X = df[features]
y = df['noncompliant_label']


X = X.dropna()
y = y.loc[X.index]  # keep y aligned with X
X_indices = X.index  # store original indices for mapping back later

print(f"Training data size after dropping NaNs: {X.shape}")

# Split X, y, and indices TOGETHER
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, X_indices, test_size=0.2, random_state=42)

# Train logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
logreg_score = logreg.predict_proba(X_test)[:, 1]

# Train random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
rf_score = rf.predict_proba(X_test)[:, 1]

# Compute ensemble average
ensemble_score = (logreg_score + rf_score) / 2

# Assign mistrust_score back to original df using idx_test
df.loc[idx_test, 'mistrust_score'] = ensemble_score

# Preview saved scores
print("Example mistrust scores:")
print(df.loc[idx_test, ['mistrust_score']].head())


Training data size after dropping NaNs: (9690, 771)
Example mistrust scores:
      mistrust_score
3879        0.181597
1576        0.108690
1746        0.161555
4860        0.052253
39          0.050795


In [ ]:
# ================================
# Save mistrust scores to file
# ================================

import os
output_dir = '/content/out_data'
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, 'mistrust_scores.csv')
# Save icustay_id and mistrust_score where mistrust_score is not null
df[['icustay_id', 'mistrust_score']].dropna().to_csv(output_file, index=False)
print(f"✅ Mistrust scores saved to {output_file}")


✅ Mistrust scores saved to /content/out_data/mistrust_scores.csv
